In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import os

In [ ]:
#O arquivo disponibilizado no site https://www.kaggle.com/datasets/nih-chest-xrays/data/data é muito grande para que seja baixado :(
#Tivemos que baixar dessa forma para funcionar, baixando diretamente do Kaggle

!mkdir -p ~/.kaggle
!echo '{"username":"KAGGLE_USERNAME","key":"KAGGLE_KEY"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia
!unzip -q chest-xray-pneumonia.zip -d /content/

# Caminhos do dataset, separando cada um dos elementos como trainamento, validação e testes
base_dir = '/content/chest_xray'
train_dir = os.path.join(base_dir, 'train')
val_dir   = os.path.join(base_dir, 'val')
test_dir  = os.path.join(base_dir, 'test')

Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
License(s): other
chest-xray-pneumonia.zip: Skipping, found more recently modified local copy (use --force to force download)
replace /content/chest_xray/__MACOSX/._chest_xray? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:

# Normalização das imagens, foram divididas por 255 para reduzir escala, ficando com 0–1
# Para poder generalizar, foram utilizadas movimentações na image, girando, ampliando, espelhando para evitar o overfitting e não virar um oráculo
#dos casos apresentados
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
# Nos conjuntos de validação e teste usamos apenas a normalização
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Geração automática de lotes de imagens com rótulos
train_data = train_datagen.flow_from_directory(
    train_dir, target_size=(150,150), batch_size=32, class_mode='binary'
)
val_data = val_datagen.flow_from_directory(
    val_dir, target_size=(150,150), batch_size=32, class_mode='binary'
)
test_data = test_datagen.flow_from_directory(
    test_dir, target_size=(150,150), batch_size=32, class_mode='binary', shuffle=False
)

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [ ]:
#Construção da Rede Neural Convolucional (CNN)
# O algoritmo de CNN é composta por camadas:
# Conv2D é utilizada para detectar padrões nas imagens(bordas, texturas)
# MaxPooling2D reduz o tamanho da imagem mantendo as informações principais
# Flatten utilizado para transformar o mapa de características em vetor
# Dense é utilizado para trabalhar as camadas densas (neurônios da rede neural)
# Dropout faz com que desative aleatoriamente neurônios, para novamente evitar overfitting
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])


# Compila o modelo definindo:
# Otimizador Adam é utilizado para ajustar pesos de forma automática
# A função de perda utilziada foi Binary Crossentropy
# Métrica utilizada foi a Acurácia
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     4,735,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,828,481 (18.42 MB)

 Trainable params: 4,828,481 (18.42 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# O modelo é treinado por 10 vezes, utilizando o conjunto de treinamento
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 278s 2s/step - accuracy: 0.7583 - loss: 0.5089 - val_accuracy: 0.6250 - val_loss: 0.9108
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 270s 2s/step - accuracy: 0.8505 - loss: 0.3371 - val_accuracy: 0.7500 - val_loss: 0.6323
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 266s 2s/step - accuracy: 0.8695 - loss: 0.2928 - val_accuracy: 0.6875 - val_loss: 0.4456
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 272s 2s/step - accuracy: 0.8888 - loss: 0.2649 - val_accuracy: 0.7500 - val_loss: 0.7974
Epoch 5/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 273s 2s/step - accuracy: 0.8945 - loss: 0.2566 - val_accuracy: 0.7500 - val_loss: 0.5646
Epoch 6/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 306s 2s/step - accuracy: 0.8929 - loss: 0.2554 - val_accuracy: 0.6250 - val_loss: 0.6029
Epoch 7/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 254s 2s/step - accuracy: 0.9036 - loss: 0.2303 - val_accuracy: 0.8125 - val_loss: 0.4963
Epoch 8/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 250s 2s/step - accuracy: 0.9093 - loss: 0.2321 - val_accu

In [ ]:
#Parte da avaliação do modelo com as imagens não utilizadas

loss, acc = model.evaluate(test_data)
print(f"Acurácia no conjunto de teste: {acc*100:.2f}%")#imprimir a acurácia

# Pra podemos mostrar a evolução da acurácia durante o treino e a validação
plt.plot(history.history['accuracy'], label='Treino')
plt.plot(history.history['val_accuracy'], label='Validação')
plt.legend()
plt.title('Evolução da acurácia')
plt.show()


In [ ]:

# Matriz de confusão pra mostrar quantas imagens foram classificadas de forma correta ou incorreta
# Colocamos para apresentar a precisão, recall e F1-score para cada uma das classes
import numpy as np
y_pred = (model.predict(test_data) > 0.5).astype("int32")
cm = confusion_matrix(test_data.classes, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Normal', 'Pneumonia'],
            yticklabels=['Normal', 'Pneumonia'])
plt.xlabel('Predito')
plt.ylabel('Real')
plt.show()

print(classification_report(test_data.classes, y_pred, target_names=['Normal','Pneumonia']))